### 1. IMPORT

In [ ]:
import os
import json

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import MultiModalRelevance, MultiModalFaithfulness
from ragas import EvaluationDataset

from multimodal_rag import MultimodalRAG
from config import OPENAI_API_KEY
from multimodal_embedder import create_embedder
from multimodal_rag import MultimodalRAG
from pdf_to_qa import generate_qa_for_pdf

### 2. Configuration

In [ ]:
PDF_FILE = "knowledge/subset_monetary_policy_report.pdf"
EMBEDDER = "ColPali"

rag = MultimodalRAG(pdf_file=PDF_FILE, embedder=create_embedder(EMBEDDER), page_mode="image_only")
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini")) # For Ragas evaluation
embeddings = OpenAIEmbeddings()

### Generate Dataset

In [ ]:
# Check if QA file already exists
qa_filepath = "QA_" + os.path.basename(PDF_FILE).replace('.pdf', '.json')

if os.path.exists(qa_filepath):
    qa_path = qa_filepath
    print(f"Using existing QA file: {qa_path}")
else:
    qa_path = generate_qa_for_pdf(PDF_FILE)
    print(f"Generated new QA file: {qa_path}")

### 3. Answering the QA

In [ ]:
with open(qa_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)
    
# Generate dataset
rag_generated_answers = []

# Check if generated answers file already exists
generated_answer_filepath = "rag_generated_answers_" + os.path.basename(qa_path)

if os.path.exists(generated_answer_filepath):
    rag_generated_answers = json.load(open(generated_answer_filepath, 'r', encoding='utf-8'))
else:
    for qa in qa_data:
        query = qa["question"]
        reference = qa["answer"]

        relevant_docs = rag.get_most_relevant_docs(query)
        response = rag.generate_answer(query, relevant_docs)
        rag_generated_answers.append(
            {
                "user_input":query,
                "retrieved_contexts":relevant_docs,
                "response":response,
                "reference":reference
            }
        )

    # Save the dataset to a JSON file
    output_dataset_file = "rag_generated_answers_" + os.path.basename(qa_path)

    with open(output_dataset_file, 'w', encoding='utf-8') as f:
        json.dump(rag_generated_answers, f, ensure_ascii=False, indent=4)

In [ ]:
for i, entry in enumerate(rag_generated_answers):
    print(f"Entry {i+1}:")
    print(f"  User Input: {entry['user_input']}")
    print(f"  Retrieved Contexts: {entry['retrieved_contexts']}")
    print(f"  Response: {entry['response']}")
    print(f"  Reference: {entry['reference']}")
    print("-" * 40)

### Generation Evaluation

In [ ]:
processed_rag_generated_answers = []

for sample in rag_generated_answers:
    processed_sample = sample.copy()
    processed_sample["retrieved_contexts"] = [
        context["content"] for context in sample["retrieved_contexts"]
    ]
    processed_rag_generated_answers.append(processed_sample)

evaluation_dataset = EvaluationDataset.from_list(processed_rag_generated_answers)

result = evaluate(dataset=evaluation_dataset,metrics=[MultiModalFaithfulness(), MultiModalRelevance()],llm=evaluator_llm)
result